<a href="https://colab.research.google.com/github/Sameersah/decision-trees-ensemble/blob/main/Decision_Tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Decision Tree Implementation

Decision trees split data recursively to minimize impurity (e.g., Gini index or entropy).

Steps:
Calculate Impurity: Use criteria like Gini impurity or entropy.
Split Data: Find the best feature and threshold to split the data.
Build Tree: Recursively split until a stopping condition (e.g., max depth or no improvement).
Make Predictions: Traverse the tree for predictions.

In [ ]:
# Decision Tree Implementation from Scratch

import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification, make_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error
from typing import List, Optional

class Node:
    """
    Node class representing a decision tree node
    """
    def __init__(self, feature_index=None, threshold=None,
                 left=None, right=None, value=None, info_gain=None):
        # For decision/split nodes
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.info_gain = info_gain

        # For leaf nodes
        self.value = value

class DecisionTree:
    """
    Decision Tree implementation for both classification and regression
    """
    def __init__(self, max_depth=None, min_samples_split=2,
                 task_type='classification'):
        """
        Initialize Decision Tree

        Args:
            max_depth (int): Maximum depth of the tree
            min_samples_split (int): Minimum samples required to split a node
            task_type (str): 'classification' or 'regression'
        """
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.task_type = task_type
        self.root = None

    def _entropy(self, y):
        """
        Calculate entropy for classification

        Args:
            y (np.array): Target labels

        Returns:
            float: Entropy value
        """
        _, counts = np.unique(y, return_counts=True)
        probabilities = counts / len(y)
        return -np.sum(probabilities * np.log2(probabilities + 1e-10))

    def _variance(self, y):
        """
        Calculate variance for regression

        Args:
            y (np.array): Target values

        Returns:
            float: Variance value
        """
        return np.var(y)

    def _information_gain(self, parent, left_child, right_child):
        """
        Calculate information gain for a split

        Args:
            parent (np.array): Parent node samples
            left_child (np.array): Left child node samples
            right_child (np.array): Right child node samples

        Returns:
            float: Information gain
        """
        # Weight of each child
        weight_left = len(left_child) / len(parent)
        weight_right = len(right_child) / len(parent)

        if self.task_type == 'classification':
            # Entropy-based information gain
            parent_impurity = self._entropy(parent)
            left_impurity = self._entropy(left_child)
            right_impurity = self._entropy(right_child)

            return parent_impurity - (weight_left * left_impurity +
                                      weight_right * right_impurity)
        else:
            # Variance reduction for regression
            parent_impurity = self._variance(parent)
            left_impurity = self._variance(left_child)
            right_impurity = self._variance(right_child)

            return parent_impurity - (weight_left * left_impurity +
                                      weight_right * right_impurity)

    def _best_split(self, X, y):
        """
        Find the best feature and threshold to split on

        Args:
            X (np.array): Feature matrix
            y (np.array): Target values

        Returns:
            tuple: Best feature index, threshold, and information gain
        """
        best_gain = -float('inf')
        best_feature = None
        best_threshold = None

        n_samples, n_features = X.shape

        for feature_index in range(n_features):
            # Get unique thresholds
            thresholds = np.unique(X[:, feature_index])

            for threshold in thresholds:
                # Split data
                left_mask = X[:, feature_index] <= threshold
                right_mask = ~left_mask

                # Skip if split is too small
                if (np.sum(left_mask) < self.min_samples_split or
                    np.sum(right_mask) < self.min_samples_split):
                    continue

                # Calculate information gain
                left_y = y[left_mask]
                right_y = y[right_mask]
                gain = self._information_gain(y, left_y, right_y)

                # Update best split if needed
                if gain > best_gain:
                    best_gain = gain
                    best_feature = feature_index
                    best_threshold = threshold

        return best_feature, best_threshold, best_gain

    def _grow_tree(self, X, y, depth=0):
        """
        Recursively grow the decision tree

        Args:
            X (np.array): Feature matrix
            y (np.array): Target values
            depth (int): Current tree depth

        Returns:
            Node: Root of the subtree
        """
        n_samples, n_features = X.shape

        # Stopping criteria
        if (self.max_depth is not None and depth >= self.max_depth or
            len(np.unique(y)) == 1 or
            n_samples < self.min_samples_split):

            # Leaf node value (mode for classification, mean for regression)
            leaf_value = (np.bincount(y).argmax() if self.task_type == 'classification'
                          else np.mean(y))
            return Node(value=leaf_value)

        # Find best split
        best_feature, best_threshold, best_gain = self._best_split(X, y)

        # No good split found
        if best_feature is None:
            leaf_value = (np.bincount(y).argmax() if self.task_type == 'classification'
                          else np.mean(y))
            return Node(value=leaf_value)

        # Create split
        left_mask = X[:, best_feature] <= best_threshold
        right_mask = ~left_mask

        # Recursively grow subtrees
        left_subtree = self._grow_tree(X[left_mask], y[left_mask], depth + 1)
        right_subtree = self._grow_tree(X[right_mask], y[right_mask], depth + 1)

        # Return internal node
        return Node(
            feature_index=best_feature,
            threshold=best_threshold,
            left=left_subtree,
            right=right_subtree,
            info_gain=best_gain
        )

    def fit(self, X, y):
        """
        Fit the decision tree

        Args:
            X (np.array): Feature matrix
            y (np.array): Target values

        Returns:
            self: Fitted decision tree
        """
        # Ensure y is integer for classification
        if self.task_type == 'classification':
            y = y.astype(int)

        # Grow the tree
        self.root = self._grow_tree(X, y)
        return self

    def _predict_single(self, x):
        """
        Predict for a single sample

        Args:
            x (np.array): Single sample

        Returns:
            Predicted value
        """
        node = self.root
        while node.value is None:
            # Traverse left or right based on feature threshold
            if x[node.feature_index] <= node.threshold:
                node = node.left
            else:
                node = node.right

        return node.value

    def predict(self, X):
        """
        Predict for multiple samples

        Args:
            X (np.array): Feature matrix

        Returns:
            np.array: Predictions
        """
        return np.array([self._predict_single(x) for x in X])

def demonstrate_decision_tree_classification():
    """
    Demonstrate Decision Tree for Classification
    """
    # Generate synthetic classification dataset
    X, y = make_classification(
        n_samples=500,
        n_features=10,
        n_informative=5,
        n_redundant=2,
        random_state=42
    )

    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    # Train Decision Tree
    clf = DecisionTree(max_depth=5, task_type='classification')
    clf.fit(X_train, y_train)

    # Predict and evaluate
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)

    print(f"Classification Accuracy: {accuracy * 100:.2f}%")

    return clf, X_test, y_test

def demonstrate_decision_tree_regression():
    """
    Demonstrate Decision Tree for Regression
    """
    # Generate synthetic regression dataset
    X, y = make_regression(
        n_samples=500,
        n_features=10,
        noise=0.1,
        random_state=42
    )

    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    # Train Decision Tree
    reg = DecisionTree(max_depth=5, task_type='regression')
    reg.fit(X_train, y_train)

    # Predict and evaluate
    y_pred = reg.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)

    print(f"Regression Mean Squared Error: {mse:.4f}")

    return reg, X_test, y_test

# Demonstrate both classification and regression
print("Classification Example:")
clf, X_test_clf, y_test_clf = demonstrate_decision_tree_classification()

print("\nRegression Example:")
reg, X_test_reg, y_test_reg = demonstrate_decision_tree_regression()

import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification, make_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error
from typing import List, Optional

# ... (Your existing DecisionTree and other functions) ...

def plot_decision_boundary_2d(X, y, model):
    """
    Plot decision boundaries for 2D datasets

    Args:
        X (np.array): Feature matrix (first 2 features used)
        y (np.array): Target values
        model (DecisionTree): Trained decision tree
    """
    plt.figure(figsize=(10, 6))

    # Use only the first 2 features for visualization
    X_2d = X[:, :2]  # Select the first 2 columns

    # Plot the feature space
    x_min, x_max = X_2d[:, 0].min() - 1, X_2d[:, 0].max() + 1
    y_min, y_max = X_2d[:, 1].min() - 1, X_2d[:, 1].max() + 1
    xx, yy = np.meshgrid(
        np.arange(x_min, x_max, 0.1),
        np.arange(y_min, y_max, 0.1)
    )

    # Predict for the entire grid (using only 2 features)
    # Create a temporary dataset with only 2 features
    grid_data = np.c_[xx.ravel(), yy.ravel()]

    # Predict using the original model but with only the first 2 features
    Z = model.predict(grid_data)
    Z = Z.reshape(xx.shape)

    # Plot decision boundary
    plt.contourf(xx, yy, Z, alpha=0.4)
    plt.scatter(X_2d[:, 0], X_2d[:, 1], c=y, alpha=0.8)
    plt.title("Decision Tree Boundary (2D)")
    plt.xlabel("Feature 1")
    plt.ylabel("Feature 2")
    plt.show()

# Example usage of decision boundary plot (for low-dimensional data)
print("\nDecision Boundary Visualization:")
if clf is not None and X_test_clf is not None and y_test_clf is not None:
    if X_test_clf.shape[1] >= 2:
        plot_decision_boundary_2d(X_test_clf, y_test_clf, clf)